# Import

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
import pandas as pd
from types import SimpleNamespace

# Hyperparameter

In [2]:
config = {
    "learning_rate": 2e-5,
    "epoch": 10,
    "batch_size": 64
}

CFG = SimpleNamespace(**config)

# Load Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_df = pd.read_csv("C:/Users/KimDongyoung/Desktop/GBT해커톤/GBT_hackerton/code/dongyoung/train_df.csv")
test_df = pd.read_csv("C:/Users/KimDongyoung/Desktop/GBT해커톤/GBT_hackerton/code/dongyoung/test_df.csv")

In [6]:
train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54609 entries, 0 to 54608
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      54609 non-null  object
 1   분류      54609 non-null  object
 2   제목      54609 non-null  object
 3   키워드     54609 non-null  object
dtypes: object(4)
memory usage: 1.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23405 entries, 0 to 23404
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      23405 non-null  object 
 1   분류      0 non-null      float64
 2   제목      23405 non-null  object 
 3   키워드     23405 non-null  object 
dtypes: float64(1), object(3)
memory usage: 731.5+ KB


# Load Model

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')     # model과 tokenizer는 항상 mapping 관계여야 한다
model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=len(train_df['분류'].unique())).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
if torch.cuda.is_available():
    print(True)
else:
    print(False)

True


# Custom Dataset

In [8]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128): # 클래스 초기화 (__init__ 메서드)
        self.texts = texts                                      # texts: 텍스트 데이터의 리스트입니다.
        self.labels = labels                                    # labels: 각 텍스트에 대한 레이블(정답)의 리스트입니다. 레이블이 없는 경우를 위해 기본값을 None으로 설정할 수 있습니다.
        self.tokenizer = tokenizer                              # tokenizer: 텍스트를 토큰화하는 데 사용할 토크나이저 객체입니다. 주로 BERT와 같은 모델의 토크나이저를 사용합니다.
        self.max_len = max_len                                  # max_len: 토큰화된 텍스트의 최대 길이입니다. 기본값은 128로 설정되어 있습니다.

    def __len__(self):                                          # 데이터셋 길이 (__len__ 메서드)
        return len(self.texts)                                  # 데이터셋의 크기를 반환합니다. 즉, 텍스트의 개수를 반환합니다.

    def __getitem__(self, item):                                # 데이터 항목 가져오기 (__getitem__ 메서드)
        text = str(self.texts[item])                            # item: 데이터셋에서 가져올 특정 인덱스입니다.
        label = self.labels[item] if self.labels is not None else -1 # label: 해당 인덱스의 레이블을 가져옵니다. 레이블이 없으면 -1로 설정합니다.
        encoding = self.tokenizer.encode_plus(                  # encoding: 토크나이저의 encode_plus 메서드를 사용하여 텍스트를 인코딩합니다. 이 과정에서 다음과 같은 옵션을 설정합니다:
            text,                                              #
            add_special_tokens=True,                            # add_special_tokens=True: 모델에 필요한 특별한 토큰을 추가합니다.
            max_length=self.max_len,                            # max_length=self.max_len: 최대 길이를 설정합니다.
            return_token_type_ids=False,                        #
            padding='max_length',                               # padding='max_length': 최대 길이에 맞춰 패딩을 추가합니다.
            truncation=True,                                    # truncation=True: 최대 길이를 초과하는 경우 잘라냅니다.
            return_attention_mask=True,                         # return_attention_mask=True: 어텐션 마스크도 반환합니다.
            return_tensors='pt',                                # return_tensors='pt': PyTorch 텐서 형식으로 반환합니다.
        )
        return {                                                # 반환하는 딕셔너리에는 원본 텍스트, 인코딩된 입력 ID, 어텐션 마스크, 레이블이 포함
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


# Data Preprocessing

In [9]:
# 데이터 준비
# train_df['키워드'] = train_df['제목'] + ' ' + train_df['키워드']
# test_df['키워드'] = test_df['제목'] + ' ' + test_df['키워드']

# 레이블 인코딩
label_encoder = {label: i for i, label in enumerate(train_df['분류'].unique())}
train_df['label'] = train_df['분류'].map(label_encoder)

# 데이터 분할 (train -> train + validation)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['분류'], random_state=42)

# 데이터셋 생성
train_dataset = TextDataset(train_df.키워드.tolist(), train_df.label.tolist(), tokenizer)   # train_df에서 키워드와 label을 리스트로 변환하여 TextDataset 클래스의 인스턴스를 생성
val_dataset = TextDataset(val_df.키워드.tolist(), val_df.label.tolist(), tokenizer)         # val_df에서 키워드와 label을 사용하여 검증 데이터셋을 생성
test_dataset = TextDataset(test_df.키워드.tolist(), None, tokenizer)                        # test_df에서 키워드를 리스트로 변환하여 TextDataset을 생성합니다. 이 데이터셋은 레이블이 없으므로 None을 전달

# 데이터 로더 생성, DataLoader는 데이터를 미니 배치 단위로 나누어서 제공해주는 역할
train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False)

In [10]:
train_dataset[3]

{'text': '용인,투기,경기도청,팀장,민간시설,편의점,투자,경기도,용인시,원삼면,SK,하이닉스,반도체,클러스터,예정지,인근,토지,매입,경기도청,팀장,투자유치,민간업체,퇴직,투자,확인,경기도,반도체,클러스터,투기,의혹,혐의,부패,방지,위반,경찰,고발,경기도,김씨,투자,진흥과,근무,민간,투자,유치,업무,전반,조사,김씨,클러스터,용인,반도체,프리미엄,아웃렛,테마파크,산업단지,업무,투자,유치,10년,경기도,조사,비위,주목,취재,본지,종합,김씨,아내,대표,가족회사,호연산업,고색동,경기도,수원시,권선구,자동차,복합,쇼핑몰,도이치오토월드,상가,지하,15평,분양,등기부,등본,거래가액,3억,신고,금융기관,4억,근저당권,설정,편의점,입점,운영,김씨,가족,호연산업,수원지점,법인등기부,등재,별개,김씨,지상,편의점,개점,준비,김씨,원삼면,토지,호연산업,명의,매입,준공,도이치오토월드,규모,최대,자동차,복합,단지,지상,규모,지하,1만,전시,시설,신차,중고차,거래업체,입주,김씨,경기도,투자,진흥과,근무,사업,착공,착공,경기도,보도자료,김씨,도이치오토월드,관계자,경기도,팀장,경기도,퇴직,김씨,누이,편의점,분양,계약,코로나,근린생활시설,분양,상태,특혜,김씨,경기도,투자,진흥,팀장,재직,가족,회사,명의,SK하이닉스,120조,투자,클러스터,예정지,용인시,처인구,원삼면,반도체,클러스터,토지,폐가,대출,포함,5억,무더기,매입,매입,시점,투자,유치,실투자금,매입,가격,토지,도면,공개,25억,김씨,퇴직,컨설턴트,투자,유치,전문,활동',
 'input_ids': tensor([   2,    0,   46,    0,   46,    0,   46, 7679,   46,    0,   46,    0,
           46, 7645,   46,    0,   46,    0,   46,    0,   46,  346,   46, 7817,
           46, 6287,   46,    0,   46,    0,   46,    0,   46,    0,   46,    0,
      

위의 텐서는 자연어 처리에서 토큰화된 텍스트를 나타내며, 각 숫자는 특정 토큰에 대한 ID를 의미한다. 출력된 텐서는 모델 학습이나 예측에 사용한다.

In [11]:
# 옵티마이저 및 학습 파라미터 설정
optimizer = AdamW(model.parameters(), lr=CFG.learning_rate)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:
# # 학습
# model.train()
# for epoch in range(CFG.epoch):
#     for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{CFG.epoch}'):
#         optimizer.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()

#     # Validation
#     model.eval()
#     val_predictions = []
#     val_true_labels = []
#     with torch.no_grad():
#         for batch in tqdm(val_loader, desc='Validating'):
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['labels'].to(device)
#             outputs = model(input_ids, attention_mask=attention_mask)
#             _, preds = torch.max(outputs.logits, dim=1)
#             val_predictions.extend(preds.cpu().tolist())
#             val_true_labels.extend(labels.cpu().tolist())

#     # 검증 결과 출력
#     val_f1 = f1_score(val_true_labels, val_predictions, average='macro')
#     print(f"Validation F1 Score: {val_f1:.2f}")

Validating: 100%|██████████| 171/171 [01:02<00:00,  2.74it/s]


Validation F1 Score: 0.20


Validating: 100%|██████████| 171/171 [01:02<00:00,  2.74it/s]


Validation F1 Score: 0.29


Validating: 100%|██████████| 171/171 [01:02<00:00,  2.73it/s]


...

Validation F1 Score: 0.48


Validating: 100%|██████████| 171/171 [01:02<00:00,  2.74it/s]


Validation F1 Score: 0.50


Validating: 100%|██████████| 171/171 [01:02<00:00,  2.73it/s]

Validation F1 Score: 0.50


In [ ]:
import numpy as np

# 초기 설정
best_f1 = -np.inf  # 가장 좋은 F1 점수 초기화
patience = 3  # 개선이 없을 경우 멈추기 전 허용할 에포크 수
counter = 0  # 개선이 없었던 에포크 수 카운터

model.train()
for epoch in range(CFG.epoch):
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{CFG.epoch}'):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_predictions = []
    val_true_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            val_predictions.extend(preds.cpu().tolist())
            val_true_labels.extend(labels.cpu().tolist())

    # 검증 결과 출력
    val_f1 = f1_score(val_true_labels, val_predictions, average='macro')
    print(f"Validation F1 Score: {val_f1:.2f}")

    # F1 점수가 개선되었는지 확인
    if val_f1 > best_f1:
        best_f1 = val_f1
        counter = 0  # 개선이 있었으므로 카운터 초기화
    else:
        counter += 1  # 개선이 없으므로 카운터 증가

    # 개선이 없었던 에포크 수가 patience를 초과하면 학습 중단
    if counter >= patience:
        print("Early stopping triggered.")
        break

# Inference

In [14]:
# 테스트 세트 추론
model.eval()
test_predictions = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        test_predictions.extend(preds.cpu().tolist())

# 라벨 디코딩
label_decoder = {i: label for label, i in label_encoder.items()}
decoded_predictions = [label_decoder[pred] for pred in test_predictions]

Testing: 100%|██████████| 366/366 [02:14<00:00,  2.71it/s]


# Submission

In [15]:
submission = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GBT해커톤/Data/sample_submission.csv")
submission["분류"] = decoded_predictions

submission.to_csv("/content/drive/MyDrive/Colab Notebooks/GBT해커톤/submission/0925_kobert.csv", encoding='UTF-8-sig', index=False)

In [16]:
submission['분류'].value_counts()

,count
분류,
지역,12402
경제:부동산,1468
사회:사건_사고,1232
경제:반도체,888
사회:사회일반,488
사회:교육_시험,408
사회:의료_건강,381
스포츠:올림픽_아시안게임,372
정치:국회_정당,370
